In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
   ━━━━━━━━

In [2]:
# !pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/Cosine-similarity.git

  Cloning https://github.com/Shannu3766/Cosine-similarity.git to /tmp/pip-req-build-t27_j4ok
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/Cosine-similarity.git /tmp/pip-req-build-t27_j4ok
  Resolved https://github.com/Shannu3766/Cosine-similarity.git to commit 76574c3690a5ecf452ca0e2bfb25b34b38b6bec5
  Preparing metadata (setup.py) ... done
  Created wheel for adalora_bi_eq13: filename=adalora_bi_eq13-0.1.0-py3-none-any.whl size=10519 sha256=ef3d4b818119073faa5a5d59f2205fe425a7d42d93ffbbdaa4fc123006b6aae7
  Stored in directory: /tmp/pip-ephem-wheel-cache-3sbu1kcb/wheels/76/bb/9e/d30494391cfeaff12e87f494af145e11c56eaec39eff965930
Successfully built adalora_bi_eq13


In [3]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
# logging.set_verbosity_error()

In [4]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
from adaptive_lora_gradient.callbacks import AdaptiveLoRACallback
import numpy as np
print("torch:", torch.__version__)
print("cuda devices:", torch.cuda.device_count())

2025-12-11 15:42:44.316163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765467764.693529      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765467764.804358      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

torch: 2.6.0+cu124
cuda devices: 2


In [5]:
import os
import warnings
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    logging
)
from peft import get_peft_model, LoraConfig
import evaluate

In [6]:
# Silence all unnecessary outputs
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
logging.set_verbosity_error()


In [7]:
from datasets import load_dataset
import time

def load_dataset_with_retry(path, name=None, max_retries=None, wait=2):
    """
    Repeatedly tries to load a dataset until it succeeds.
    - max_retries: None → infinite retries
    - wait: base wait time (exponential backoff)
    """
    attempt = 0

    while True:
        try:
            if name is not None:
                ds = load_dataset(path, name)
            else:
                ds = load_dataset(path)

            print(f"Dataset loaded successfully after {attempt} attempts.")
            return ds

        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  # exponential cap

            print(f"[Attempt {attempt}] Failed to load dataset: {e}")
            print(f"Retrying in {wait_time} seconds...\n")

            # If user gives a max_retries limit
            if max_retries is not None and attempt >= max_retries:
                print("Max retries reached. Raising error.")
                raise e

            time.sleep(wait_time)
dataset = load_dataset_with_retry("glue", "sst2")


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset loaded successfully after 0 attempts.


In [8]:
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

In [9]:
print("Model stareted Loading")

Model stareted Loading


In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    device_map="auto"
)

In [11]:
print("model Loaded")

model Loaded


In [12]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# 4️⃣ Load tokenizer and fix pad token
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# TinyLlama / LLaMA-style tokenizer does NOT need add_prefix_space
# (that's mostly for GPT-2-like models)
# So we don't pass add_prefix_space=True here.

if tokenizer.pad_token is None:
    # For LLaMA-like models, use eos_token as pad_token
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

MAX_LEN = 128  # SST-2 sentences are short; 128 is usually plenty


def clean_text(s):
    if s is None:
        return ""
    # strip whitespace + collapse multiple spaces/newlines
    return " ".join(str(s).strip().split())


def tokenize_function(examples):
    # 🧠 Make it more "instructional" for a causal LM
    texts = [
        f"Review: {clean_text(x)}\nSentiment:"
        for x in examples["sentence"]
    ]

    return tokenizer(
        texts,
        truncation=True,
        padding=False,   # ✅ dynamic padding via DataCollator
        max_length=MAX_LEN,
    )


# Remove raw text + idx, keep label
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["sentence", "idx"],
)

# Trainer expects "labels" by default
if "label" in tokenized_dataset["train"].column_names:
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

tokenized_dataset.set_format("torch")

# 6️⃣ Data collator handles dynamic padding
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    pad_to_multiple_of=8,  
)


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [13]:
accuracy = evaluate.load("accuracy")
rank = 8
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)}

# 8️⃣ LoRA config
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=rank,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ]
)
model = get_peft_model(model, peft_config)


In [14]:
from peft.tuners.lora import LoraLayer

def get_lora_module_names(peft_model):
    names = []
    for name, module in peft_model.named_modules():
        if isinstance(module, LoraLayer):
            names.append(name)
    return names

lora_names = get_lora_module_names(model)   # `model` is your PEFT-wrapped model
print(f"🔢 Number of LoRA modules: {len(lora_names)}\n")

# for n in lora_names:
    # print(" ", n)
num_lora_modules = len(lora_names)
TOTAL_RANK_BUDGET = num_lora_modules*rank  # example

avg_rank_per_module = TOTAL_RANK_BUDGET / num_lora_modules
print(
    f"💡 With TOTAL_RANK_BUDGET={TOTAL_RANK_BUDGET} over "
    f"{num_lora_modules} modules, avg rank ≈ {avg_rank_per_module:.2f}"
)


🔢 Number of LoRA modules: 154

💡 With TOTAL_RANK_BUDGET=1232 over 154 modules, avg rank ≈ 8.00


In [15]:
training_args = TrainingArguments(
    disable_tqdm=False,
    output_dir="./SST2_adaptive_results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    logging_steps=500,
    eval_strategy="epoch",
    eval_steps=500,
    bf16=True,              
    report_to="none",
    save_strategy="no",
)


In [16]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [17]:

from torch.utils.data import DataLoader
train_dataset = tokenized_dataset["train"].select(range(10000))
eval_dataset = tokenized_dataset["validation"]

val_dataloader = DataLoader(
    eval_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator
)

In [18]:
# from adaptive_lora.callbacks import AdaptiveLoRACallback


adaptive_callback = AdaptiveLoRACallback(
    total_rank=TOTAL_RANK_BUDGET,
    val_dataloader=val_dataloader,
    score_smoothing_beta=0.8,   
    update_interval=2,        
    warmup_epochs=0,    
    cooldown_epochs=1,  
    min_rank=4,
    tau=0.7
)


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback],
)

In [20]:
trainer.train()

⏭️ AdaptiveLoRA: Interval skip (Epoch 1). Keeping ranks.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.215300,0.211419,{'accuracy': 0.9461009174311926}
2,0.135000,0.250599,{'accuracy': 0.9495412844036697}
3,0.142800,0.260351,{'accuracy': 0.9495412844036697}



--- AdaptiveLoRA: Adapting Ranks for Epoch 2 ---


Computing Importance:   0%|          | 0/109 [00:00<?, ?it/s]

Rank Updates (Epoch 2):
  - base_model.model.model.layers.0.self_attn.q_proj: r=8 → 6 (Score: 0.2367)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 0.3829)
  - base_model.model.model.layers.0.self_attn.v_proj: r=8 (Unchanged, Score: 0.3968)
  - base_model.model.model.layers.0.self_attn.o_proj: r=8 → 13 (Score: 0.7409)
  - base_model.model.model.layers.0.mlp.gate_proj: r=8 → 9 (Score: 0.5007)
  - base_model.model.model.layers.0.mlp.up_proj: r=8 → 9 (Score: 0.5053)
  - base_model.model.model.layers.0.mlp.down_proj: r=8 → 9 (Score: 0.5053)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 → 7 (Score: 0.3531)
  - base_model.model.model.layers.1.self_attn.k_proj: r=8 → 14 (Score: 0.7950)
  - base_model.model.model.layers.1.self_attn.v_proj: r=8 → 11 (Score: 0.6163)
  - base_model.model.model.layers.1.self_attn.o_proj: r=8 → 10 (Score: 0.5881)
  - base_model.model.model.layers.1.mlp.gate_proj: r=8 → 10 (Score: 0.5357)
  - base_model.model.model.layers.1

TrainOutput(global_step=1875, training_loss=0.15515695190429687, metrics={'train_runtime': 3549.2665, 'train_samples_per_second': 8.452, 'train_steps_per_second': 0.528, 'total_flos': 8733261920600064.0, 'train_loss': 0.15515695190429687, 'epoch': 3.0})

In [21]:
met = trainer.evaluate()
print(met)

{'eval_loss': 0.26035064458847046, 'eval_accuracy': {'accuracy': 0.9495412844036697}, 'eval_runtime': 49.7052, 'eval_samples_per_second': 17.543, 'eval_steps_per_second': 2.193, 'epoch': 3.0}
